# Exploration of ESPN API

We are interested in defining "Baselines" for four decision criteria: Draft, Waiver Wire, Start/Sit, Trade. 

Ideal Baselines: 
* Draft Baseline = Points Added by Autodrafting
* Waiver Wire Baseline = Average points Added per waiver addition
* Trade = Assume No Trade as Baseline
* Start/Sit = Points Added by Using Highest ESPN Projected Lineup

Before we can fully commit to these data points, we need to determine if they are actually available in our dataset. We will start by pulling some information about the autodraft tags/data. 

In [ ]:
import requests
from collections import Counter

BASE = "https://lm-api-reads.fantasy.espn.com/apis/v3/games/ffl"

def fetch_draft(league_id: int, season: int):
    url = f"{BASE}/seasons/{season}/segments/0/leagues/{league_id}?view=mDraftDetail"
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    return r.json()

def debug_autodraft_fields(league_id: int, season: int, n=10):
    data = fetch_draft(league_id, season)
    picks = data.get("draftDetail", {}).get("picks", [])
    if not picks:
        print("No picks found. (League may be auction/keeper edge case, private, or draft not completed.)")
        return

    print(f"\nLeague {league_id} season {season}: {len(picks)} picks")

    # Show keys present in pick objects (first pick)
    print("\nPick keys (first pick):")
    print(sorted(picks[0].keys()))

    # Show any keys that include 'auto' anywhere
    auto_keys = sorted({k for p in picks for k in p.keys() if "auto" in k.lower()})
    print("\nKeys containing 'auto':", auto_keys)

    # Show unique values for common candidates
    candidates = ["autoDraftType", "autodraftType", "isAuto", "autoPick", "pickType", "draftType", "pickedBy"]
    for key in candidates:
        vals = [p.get(key) for p in picks if key in p]
        if vals:
            print(f"\nUnique values for '{key}':", sorted(set(vals), key=lambda x: str(x)))

    # Also print the first few picks verbatim (trimmed)
    print(f"\nFirst {n} picks (trimmed to auto-related fields):")
    for p in picks[:n]:
        trimmed = {k: p.get(k) for k in p.keys() if "auto" in k.lower() or k in ("teamId","pickNumber","playerId","pickedBy","pickType")}
        print(trimmed)

# Run this for ONE of your leagues first
debug_autodraft_fields(1409356, 2024)



League 1409356 season 2024: 160 picks

Pick keys (first pick):
['autoDraftTypeId', 'bidAmount', 'id', 'keeper', 'lineupSlotId', 'memberId', 'nominatingTeamId', 'overallPickNumber', 'playerId', 'reservedForKeeper', 'roundId', 'roundPickNumber', 'teamId', 'tradeLocked']

Keys containing 'auto': ['autoDraftTypeId']

First 10 picks (trimmed to auto-related fields):
{'autoDraftTypeId': 0, 'playerId': 3117251, 'teamId': 14}
{'autoDraftTypeId': 3, 'playerId': 4430807, 'teamId': 12}
{'autoDraftTypeId': 2, 'playerId': 4427366, 'teamId': 2}
{'autoDraftTypeId': 0, 'playerId': 3929630, 'teamId': 10}
{'autoDraftTypeId': 0, 'playerId': 4241389, 'teamId': 3}
{'autoDraftTypeId': 0, 'playerId': 3116406, 'teamId': 15}
{'autoDraftTypeId': 3, 'playerId': 4374302, 'teamId': 4}
{'autoDraftTypeId': 3, 'playerId': 4362628, 'teamId': 16}
{'autoDraftTypeId': 3, 'playerId': 4262921, 'teamId': 17}
{'autoDraftTypeId': 0, 'playerId': 4047646, 'teamId': 1}


Given the details that we learned from the cell above, we can pull the autodraft tags for all teams/leagues. 

In [ ]:
import requests
from collections import defaultdict
import pandas as pd

BASE_URL = "https://lm-api-reads.fantasy.espn.com/apis/v3/games/ffl"


df = pd.read_csv(r"C:\Users\dbbra\OneDrive\Documents\GitHub\fantasy_football\data\raw\espn\targets_10team_ppr1_season_2024.csv")
leagues = list(zip(df["league_id"], df["season"]))


def get_autodraft_teams(league_id, season, threshold=0.8):
    url = f"{BASE_URL}/seasons/{season}/segments/0/leagues/{league_id}?view=mDraftDetail"
    r = requests.get(url, timeout=30)
    r.raise_for_status()
    data = r.json()

    picks = data.get("draftDetail", {}).get("picks", [])
    team_flags = defaultdict(list)

    for p in picks:
        team_id = p.get("teamId")
        # Key change: autoDraftTypeId is numeric in your payload
        auto = (p.get("autoDraftTypeId", 0) != 0)
        team_flags[team_id].append(auto)

    results = {}
    for team_id, flags in team_flags.items():
        auto_picks = sum(flags)
        total = len(flags)
        auto_rate = auto_picks / total if total else 0.0
        results[team_id] = {
            "auto_picks": auto_picks,
            "total_picks": total,
            "auto_rate": round(auto_rate, 2),
            "is_autodraft_team": auto_rate >= threshold,
        }

    return results

for league_id, season in leagues:
    print(f"\nLeague {league_id} ({season})")
    try:
        teams = get_autodraft_teams(league_id, season)
        for team_id, info in teams.items():
            flag = "AUTO" if info["is_autodraft_team"] else "manual"
            print(
                f"  Team {team_id}: "
                f"{info['auto_picks']}/{info['total_picks']} auto "
                f"({info['auto_rate']*100:.0f}%) → {flag}"
            )
    except Exception as e:
        print(f"  ERROR: {e}")


Now that we can identify which teams were autodrafted, we can use that later in the analysis. This data will be key to generating our Draft Baseline. This methodology needs to be refined to account for teams that autodrafted partially. 

Next, we will see if we have to data needed for Waiver Wire Baseline. 